<a href="https://colab.research.google.com/github/riccardomarin/Tutorial_Spectral_Geometry/blob/main/inverse/02_Localization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will see how to exploit the relationship between eignvalues of the LBO under Dirichlet boundary conditions and eigenvalues of the hamiltonian operator to extract a portion of a surface.


References: 
* Rampini, Arianna, et al. "Correspondence-free region localization for partial shape similarity via hamiltonian spectrum alignment." 2019 International Conference on 3D Vision (3DV). IEEE, 2019.

Related repositories:
* https://github.com/AriannaRampini/HamiltonianSpectrumAlignment

In [ ]:
!git clone https://github.com/riccardomarin/Tutorial_Spectral_Geometry.git
%cd Tutorial_Spectral_Geometry/inverse

!pip install plyfile
# !pip install --upgrade setuptools

!pip install pymanopt
# !git clone https://github.com/pymanopt/pymanopt.git
# %cd pymanopt
# !python setup.py build
# !python setup.py install
# %cd ..

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
DEVICE = 'cpu'

In [ ]:
import sys
sys.path.append('../utils')
sys.path.append('./pymanopt')
    
import numpy as np
import torch

import matplotlib.pyplot as plt

import utils_mesh 
from utils_spectral import LB_cotan as lbo, Eigendecomposition as eigh

We will try to "localize" the shape of Heart on a portion of a plane just from its eigenvalues.

In [ ]:
from PIL import Image, ImageOps

im = ImageOps.grayscale(Image.open('../data/heart.png'))
plt.imshow(im)
print(im.size)

im = im.resize((30,30))
plt.imshow(im)
print(im.size)

We tesselate the portion of the plane with a regular grid and the canonical meshing

In [ ]:
X,Y = np.meshgrid(np.linspace(0,1,30),np.linspace(0,1,30))
VERT = torch.tensor(np.stack([X.flatten(),Y.flatten(),Y.flatten()*0],-1)).double()

#generate canonical triangulation
indexes = np.arange(X.size).reshape(X.shape)
upper_left_tri = np.stack([indexes[:-1,:-1],indexes[:-1,1:],indexes[1:,:-1]],-1).reshape(-1,3)

fig = utils_mesh.plot_colormap([VERT],[upper_left_tri],[None])
fig.show()

lower_right_tri = np.stack([indexes[:-1,1:],indexes[1:,1:],indexes[1:,:-1]],-1).reshape(-1,3)
TRIV = torch.tensor(np.concatenate([upper_left_tri,lower_right_tri],0))
fig = utils_mesh.plot_colormap([VERT],[TRIV],[None],wireframe=True)
fig.show()


Now we use the potential to compute the eigenvalues of the Laplacian corresponding to the heart with Dirichlet boundary conditions.

### Hamiltonian operator
Given a potential function $v: \mathcal{X} \rightarrow \mathbb{R}_{+}$ on the surface, the Hamiltonian can be defined in terms of the LBO operator $\Delta_{\mathcal{X}}$ and operates on a scalar function $f$ as:

\begin{align}
H_{\mathcal{X}}(v) f=\Delta_{\mathcal{X}} f+v f,
\end{align}

Eigenvalues of the (discretized) Hamiltonian can be computed solving the generalized eigenvlues problem:

\begin{align}
(\mathbf{W}+\mathbf{A} \operatorname{diag}(\mathbf{v})) \boldsymbol{\Psi}=\mathbf{A} \boldsymbol{\Psi} \operatorname{diag}(\boldsymbol{\lambda})
\end{align}

with $\Delta_{\mathcal{X}} = \mathbf{A}^{-1}\mathbf{W}$


### Relation with Dirichlet boundary conditions
Given a sub-shape $\mathcal{P}$ and an indicator function $v : \mathcal{X} \rightarrow \{0,1\}$ identifying the missing region on the full shape $\mathcal{X}$, the following equality holds:

\begin{align}
\lambda \left ( H_\mathcal{X}( v\tau ) \right ) = \lambda_{dir}\Delta_{\mathcal{P}}
\end{align}



In [ ]:
k = 30

def eig(S,invMass,k=20):
    Lnorm = invMass[:,None]*S*invMass[None,:]
    evecs, evals = eigh(Lnorm,k)
    return evals, evecs

stiff, lumped_mass = lbo(VERT,TRIV)
inv_sqrt_mass = lumped_mass.rsqrt()
evals2, evecs2 = eig(stiff,inv_sqrt_mass, k)

mask = torch.tensor(np.array(im).flatten()).double()/255
evals1,evecs1 = eig(stiff + torch.diag_embed(mask*1e9)*lumped_mass,inv_sqrt_mass, k)
evecs1 = evecs1*lumped_mass[:,None]


fig = utils_mesh.plot_colormap([VERT]*3,[TRIV]*3,[mask, evecs1[:,1], evecs1[:,2]])
fig.show()

fig = utils_mesh.plot_colormap([VERT]*3,[TRIV]*3,[mask, evecs2[:,1], evecs2[:,2]])
fig.show()
# evecs1.shape

plt.plot(evals1)
plt.plot(evals2)
plt.show()

In [ ]:
vinit = -torch.exp(-(VERT-0.5)[:,:2].norm(dim=-1)**0.5)*10# + torch.randn(VERT.shape[0])*1e-6
fig = utils_mesh.plot_colormap([VERT],[TRIV],[vinit.tanh()+1])
fig.show()

We are now ready to write down the optimization problem

In [ ]:
tau = 1e4
ham = lambda v : eig(stiff + torch.diag_embed( (v.tanh()+1)*tau)*lumped_mass, inv_sqrt_mass, k)[0]


v = torch.tensor(vinit.clone(), requires_grad=True)
optimizer = torch.optim.Adam([v], lr=1e-1)
target_evals = evals1.detach().clone()

for t in range(100):
    optimizer.zero_grad()
    
    evals = ham(v)
    loss =  torch.sum(((evals-target_evals)/(target_evals))**2)

#     torch.nn.utils.clip_grad_norm_([v], 1e-1)
    
    loss.backward()
    optimizer.step()

    if(t%10==0):
        print(loss.item())


Higher order optimization methods allows to reach better performance (even if it could take a while...)

In [ ]:
import pymanopt
import pymanopt.function
from pymanopt import Problem
from pymanopt.manifolds import Euclidean
from pymanopt.optimizers.trust_regions import TrustRegions


# A solver that involves the hessian
solver = TrustRegions(maxiter=5000,maxtime=5000)
manifold = Euclidean(v.shape[0])

@pymanopt.function.pytorch(manifold)
def cost(v):
    evals = ham(v)
    loss =  torch.sum(((evals-target_evals)/(target_evals))**2)
    return loss


# Create the problem with extra cost function arguments
problem = Problem(manifold=manifold, cost=cost)

wopt = solver.run(problem,initial_point=v.detach().numpy())

In [ ]:
v.data = torch.tensor(wopt.point)
v = torch.load('results/heart_loc.pt')

print('Final loss: %.2e' % cost(v.data.numpy()))
evals_o = ham(v)

evals_o,evecs_o = eig(stiff + torch.diag_embed( (v.tanh()+1)*tau)*lumped_mass,inv_sqrt_mass,k)
evecs_o = evecs_o*lumped_mass[:,None]

plt.plot(evals_o.data)
plt.plot(target_evals.data)
plt.show()
plt.plot((target_evals-evals_o).data)
# torch.save(v,'v.pt')

# (tanh(a)+1)*t*lm = th(v)*t + t -> a = arctanh((th(v) + 1 - lm)/lm)

In [ ]:
#show the optimized potential
fig = utils_mesh.plot_colormap([VERT]*1,[TRIV]*1,[v])
fig.show()

#compare eigenvectors:
fig = utils_mesh.plot_colormap([VERT]*3,[TRIV]*3,[evecs_o[:,1], evecs_o[:,2], evecs_o[:,10]])
fig.show()

fig = utils_mesh.plot_colormap([VERT]*3,[TRIV]*3,[evecs1[:,1], evecs1[:,2], evecs1[:,10]])
fig.show()


